In [1]:
import basedosdados as bd
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## Importing the Dataset from Base dos Dados

In [3]:
# the results of municipal elections

query = """
SELECT id_municipio, sigla_uf, cargo, sigla_partido, resultado, votos, ano
FROM basedosdados.br_tse_eleicoes.resultados_candidato
WHERE ano in (2020, 2016)
"""

results_municipal = bd.read_sql(query, 
                                billing_project_id="analise-eleitoral-330723")

Downloading: 100%|██████████| 967219/967219 [00:25<00:00, 37259.67rows/s]


In [4]:
results_municipal.head(10)

,id_candidato_bd
0,1302885
1,649297
2,1702813
3,1698446
4,1382883
5,928976
6,1040731
7,1224019
8,1277680
9,1164189


In [ ]:
# the results of municipal elections

query = """
SELECT id_municipio, sigla_uf, cargo, sigla_partido, resultado, votos, ano
FROM basedosdados.br_tse_eleicoes.resultados_candidato
WHERE ano in (2018, 2014)
"""

results_federal = bd.read_sql(query, 
                         billing_project_id="analise-eleitoral-330723")

In [ ]:
results_federal.head(10)

In [ ]:
# merging data about election results and general information about candidates 

In [2]:
# teste

query = """
SELECT c.ano, c.sigla_uf, c.id_municipio, c.cpf, id_candidato_bd, c.sigla_partido, c.cargo, r.resultado, c.idade, c.genero, c.instrucao, c.raca
FROM basedosdados.br_tse_eleicoes.candidatos as c
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato as r 
USING (id_candidato_bd)
WHERE c.ano in (2016, 2020)
"""

municipal = bd.read_sql(query, 
                        billing_project_id="analise-eleitoral-330723")


Downloading: 100%|██████████| 1985087/1985087 [03:55<00:00, 8439.54rows/s]


In [6]:
municipal = municipal.drop_duplicates()

,ano,sigla_uf,id_municipio,cpf,id_candidato_bd,sigla_partido,cargo,resultado,idade,genero,instrucao,raca
0,2016,AP,1600204,00004747208,309,PRP,vereador,suplente,40.0,masculino,ensino fundamental incompleto,branca
2,2016,AP,1600303,00014428210,1051,PSOL,vereador,suplente,28.0,masculino,ensino medio completo,parda
4,2016,AP,1600303,00035814209,2850,REPUBLICANOS,vereador,suplente,28.0,feminino,ensino superior completo,branca
5,2016,AP,1600535,00038862204,3061,MDB,vereador,nao eleito,28.0,masculino,ensino superior completo,parda
6,2016,AP,1600055,00049124200,3830,PDT,vereador,eleito por qp,28.0,masculino,ensino medio completo,branca
